In [7]:
!pip install llama-index torch transformers chromadb Ollama llama-index-llms-ollama

In [8]:
# Import modules
from llama_index.llms.ollama import Ollama
from pathlib import Path
import chromadb
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.legacy.storage.storage_context import StorageContext
from llama_index.legacy.vector_stores.chroma import ChromaVectorStore
import pandas as pd

# Load CSV data
df = pd.read_csv('./6bca48b1-fine_food_reviews.csv')

# Create Chroma DB client and store
client = chromadb.PersistentClient(path="./chroma_db_data")

# Check if the collection exists
try:
    chroma_collection = client.get_collection(name="reviews")
except chromadb.errors.CollectionNotFoundError:
    # Create the collection if it doesn't exist
    chroma_collection = client.create_collection(name="reviews")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Initialize Ollama and ServiceContext
llm = Ollama(model="mixtral")
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local")

# Create VectorStoreIndex and query engine
index = VectorStoreIndex.from_documents(df, service_context=service_context, storage_context=storage_context)
query_engine = index.as_query_engine()

# Perform a query and print the response
response = query_engine.query("What are the thoughts on food quality?")
print(response)

/tmp/ipykernel_46250/2608264543.py:28: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model="local")
/home/alysson/workspace/rag-with-chromadb-llama-index-ollama-csv/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ValueError: The `model_name` argument must be provided.

In [ ]:
# Import modules
import chromadb
from llama_index .core import VectorStoreIndex, ServiceContext
from llama_index.llms.ollama import Ollama
from llama_index.legacy.vector_stores.chroma import ChromaVectorStore

# Create Chroma DB client and access the existing vector store
client = chromadb.PersistentClient(path="./chroma_db_data")
chroma_collection = client.get_collection(name="reviews")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Initialize Ollama and ServiceContext
llm = Ollama(model="mixtral")
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local")

# Create VectorStoreIndex and query engine with a similarity threshold of 20
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context, similarity_top_k=20)
query_engine = index.as_query_engine()

# Perform a query and print the response
response = query_engine.query("What are the thoughts on food quality?")
print(response)